In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image

def select_samples_gradient(image, num_samples):
    # Check image properties
    print(f"Image shape: {image.shape}")
    print(f"Image dtype: {image.dtype}")
    
    # Convert to grayscale if the image is in color
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    elif len(image.shape) == 2:
        gray = image
    else:
        raise ValueError(f"Unexpected image shape: {image.shape}")
    
    # Ensure the image is in a format compatible with cv2.Sobel
    gray = gray.astype(np.float32)
    
    # Compute gradients
    gradient_x = cv2.Sobel(gray, cv2.CV_32F, 1, 0, ksize=3)
    gradient_y = cv2.Sobel(gray, cv2.CV_32F, 0, 1, ksize=3)
    gradient_magnitude = np.sqrt(gradient_x**2 + gradient_y**2)
    
    # Normalize gradient magnitude
    gradient_magnitude = (gradient_magnitude - gradient_magnitude.min()) / (gradient_magnitude.max() - gradient_magnitude.min())
    
    # Flatten the gradient magnitude
    flat_gradients = gradient_magnitude.flatten()
    
    # Select indices based on gradient magnitude
    selected_indices = np.random.choice(
        flat_gradients.size,
        size=num_samples,
        replace=False,
        p=flat_gradients / flat_gradients.sum()
    )
    
    # Convert to 2D indices
    h, w = gradient_magnitude.shape
    selected_y = selected_indices // w
    selected_x = selected_indices % w
    
    return np.column_stack((selected_y, selected_x))


# Load and process RGB image
try:
    rgb_image = np.array(Image.open('./data/Replica/room1/results/frame000428.jpg'))
    print("RGB Image loaded successfully")
    print(f"RGB Image shape: {rgb_image.shape}")
    print(f"RGB Image dtype: {rgb_image.dtype}")
except Exception as e:
    print(f"Error loading RGB image: {e}")
    rgb_image = None

# Load and process depth image
try:
    depth_image = np.array(Image.open('./data/Replica/room1/results/depth000428.png'))
    print("\nDepth Image loaded successfully")
    print(f"Depth Image shape: {depth_image.shape}")
    print(f"Depth Image dtype: {depth_image.dtype}")
except Exception as e:
    print(f"Error loading depth image: {e}")
    depth_image = None

# Number of samples
num_samples = 1000

# Process RGB image
if rgb_image is not None:
    try:
        rgb_samples = select_samples_gradient(rgb_image, num_samples)
        print("\nRGB image processed successfully")
    except Exception as e:
        print(f"Error processing RGB image: {e}")
        rgb_samples = None
else:
    rgb_samples = None

# Process depth image
if depth_image is not None:
    try:
        depth_samples = select_samples_gradient(depth_image, num_samples)
        print("\nDepth image processed successfully")
    except Exception as e:
        print(f"Error processing depth image: {e}")
        depth_samples = None
else:
    depth_samples = None

# Plotting function
def plot_samples(image, samples, title):
    plt.figure(figsize=(10, 10))
    plt.imshow(image, cmap='gray' if len(image.shape) == 2 else None)
    plt.scatter(samples[:, 1], samples[:, 0], c='r', s=1)
    plt.title(title)
    plt.axis('off')
    plt.show()

# Plot results
if rgb_image is not None and rgb_samples is not None:
    plot_samples(rgb_image, rgb_samples, 'Gradient-based Sampling on RGB Image')

if depth_image is not None and depth_samples is not None:
    plot_samples(depth_image, depth_samples, 'Gradient-based Sampling on Depth Image')

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image

def select_samples_gradient(image, num_samples):
    # Convert to grayscale if the image is in color
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    elif len(image.shape) == 2:
        gray = image
    else:
        raise ValueError(f"Unexpected image shape: {image.shape}")
    
    # Ensure the image is in a format compatible with cv2.Sobel
    gray = gray.astype(np.float32)
    
    # Compute gradients using Sobel
    gradient_x = cv2.Sobel(gray, cv2.CV_32F, 1, 0, ksize=3)
    gradient_y = cv2.Sobel(gray, cv2.CV_32F, 0, 1, ksize=3)
    gradient_magnitude = np.sqrt(gradient_x**2 + gradient_y**2)
    
    # Normalize gradient magnitude
    gradient_magnitude = (gradient_magnitude - gradient_magnitude.min()) / (gradient_magnitude.max() - gradient_magnitude.min())
    
    # Flatten the gradient magnitude
    flat_gradients = gradient_magnitude.flatten()
    
    # Select indices based on gradient magnitude
    selected_indices = np.random.choice(
        flat_gradients.size,
        size=num_samples,
        replace=False,
        p=flat_gradients / flat_gradients.sum()
    )
    
    # Convert to 2D indices
    h, w = gradient_magnitude.shape
    selected_y = selected_indices // w
    selected_x = selected_indices % w
    
    return np.column_stack((selected_y, selected_x)), gradient_magnitude

# Load image (works for both RGB and grayscale)
image = np.array(Image.open('./data/Replica/room1/results/frame000428.jpg'))

# Number of samples
num_samples = 1000

# Process image
samples, gradient_magnitude = select_samples_gradient(image, num_samples)

# Plotting functions
def plot_image_with_samples(image, samples, title):
    plt.figure(figsize=(10, 10))
    plt.imshow(image, cmap='gray' if len(image.shape) == 2 else None)
    plt.scatter(samples[:, 1], samples[:, 0], c='r', s=1)
    plt.title(title)
    plt.axis('off')
    plt.show()

def plot_gradient_magnitude(gradient_magnitude, title):
    plt.figure(figsize=(10, 10))
    plt.imshow(gradient_magnitude, cmap='viridis')
    plt.colorbar(label='Gradient Magnitude')
    plt.title(title)
    plt.axis('off')
    plt.show()

# Plot results
plot_image_with_samples(image, samples, 'Original Image with Sampled Points')
plot_gradient_magnitude(gradient_magnitude, 'Gradient Magnitude (Sobel)')

# Histogram of gradient magnitudes at sampled points
sampled_gradients = gradient_magnitude[samples[:, 0], samples[:, 1]]
plt.figure(figsize=(10, 5))
plt.hist(sampled_gradients, bins=50)
plt.title('Histogram of Gradient Magnitudes at Sampled Points')
plt.xlabel('Gradient Magnitude')
plt.ylabel('Frequency')
plt.show()

print(f"Mean gradient magnitude at sampled points: {sampled_gradients.mean():.4f}")
print(f"Median gradient magnitude at sampled points: {np.median(sampled_gradients):.4f}")
print(f"Overall mean gradient magnitude: {gradient_magnitude.mean():.4f}")

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image

def select_samples_gradient(image, num_samples):
    # Convert to grayscale if the image is in color
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    elif len(image.shape) == 2:
        gray = image
    else:
        raise ValueError(f"Unexpected image shape: {image.shape}")
    
    # Ensure the image is in a format compatible with cv2.Sobel
    gray = gray.astype(np.float32)
    
    # Compute gradients using Sobel
    gradient_x = cv2.Sobel(gray, cv2.CV_32F, 1, 0, ksize=3)
    gradient_y = cv2.Sobel(gray, cv2.CV_32F, 0, 1, ksize=3)
    gradient_magnitude = np.sqrt(gradient_x**2 + gradient_y**2)
    
    # Normalize gradient magnitude
    gradient_magnitude = (gradient_magnitude - gradient_magnitude.min()) / (gradient_magnitude.max() - gradient_magnitude.min())
    
    # Flatten the gradient magnitude
    flat_gradients = gradient_magnitude.flatten()
    
    # Select indices based on gradient magnitude
    selected_indices = np.random.choice(
        flat_gradients.size,
        size=num_samples,
        replace=False,
        p=flat_gradients / flat_gradients.sum()
    )
    
    # Convert to 2D indices
    h, w = gradient_magnitude.shape
    selected_y = selected_indices // w
    selected_x = selected_indices % w
    
    return np.column_stack((selected_y, selected_x)), gradient_magnitude

def plot_image_with_samples(image, samples, title):
    plt.figure(figsize=(10, 10))
    plt.imshow(image, cmap='gray' if len(image.shape) == 2 else None)
    plt.scatter(samples[:, 1], samples[:, 0], c='r', s=1)
    plt.title(title)
    plt.axis('off')
    plt.show()

def plot_gradient_magnitude(gradient_magnitude, title):
    plt.figure(figsize=(10, 10))
    plt.imshow(gradient_magnitude, cmap='viridis')
    plt.colorbar(label='Gradient Magnitude')
    plt.title(title)
    plt.axis('off')
    plt.show()

def process_and_plot(image, title_prefix, num_samples=1000):
    samples, gradient_magnitude = select_samples_gradient(image, num_samples)
    
    plot_image_with_samples(image, samples, f'{title_prefix} with Sampled Points')
    plot_gradient_magnitude(gradient_magnitude, f'{title_prefix} Gradient Magnitude (Sobel)')
    
    sampled_gradients = gradient_magnitude[samples[:, 0], samples[:, 1]]
    plt.figure(figsize=(10, 5))
    plt.hist(sampled_gradients, bins=50)
    plt.title(f'Histogram of {title_prefix} Gradient Magnitudes at Sampled Points')
    plt.xlabel('Gradient Magnitude')
    plt.ylabel('Frequency')
    plt.show()
    
    print(f"{title_prefix} Statistics:")
    print(f"Mean gradient magnitude at sampled points: {sampled_gradients.mean():.4f}")
    print(f"Median gradient magnitude at sampled points: {np.median(sampled_gradients):.4f}")
    print(f"Overall mean gradient magnitude: {gradient_magnitude.mean():.4f}")
    print("\n")

# Load RGB image
rgb_image = np.array(Image.open('./data/Replica/room1/results/frame000428.jpg'))

# Load depth image
depth_image = np.array(Image.open('./data/Replica/room1/results/depth000428.png'))

# Process and plot RGB image
process_and_plot(rgb_image, "RGB Image")

# Process and plot depth image
process_and_plot(depth_image, "Depth Image")

In [ ]:
# Load RGB image
rgb_image = np.array(Image.open('./data/Replica/room1/results/frame001428.jpg'))

# Load depth image
depth_image = np.array(Image.open('./data/Replica/room1/results/depth001428.png'))

# Process and plot RGB image
process_and_plot(rgb_image, "RGB Image")

# Process and plot depth image
process_and_plot(depth_image, "Depth Image")